In [3]:
from utils import *
import time
from sklearn.svm import LinearSVC, SVC
from copy import deepcopy
import numpy as np
import scipy.io as sio

import math
import tensorflow as tf
from scipy import signal
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import mutual_info_classif

'''
from mutual_information import mutual_information 
        https://gist.github.com/elsonidoq/4230222
        doesn't make any deffrence in output    
'''
import math
import time
import pyriemann.utils.mean as rie_mean
from scipy.signal import hilbert
from scipy import linalg

import pandas as pd

#get_data.py loads the dataset for a spceific subject, and the option of whether the training data or testing data
#utils.py have the underlying functions such as extract features function and the filter bank that allow for the FBCSP class to work
#the FBCSP class showcases the different function that work hand in hand to provied the FBCSP algorithm

#Figure out how to utilize the FBCSP Class
#Include code from Utils.py and COMMENTS
#Test classifier
#Retrieve Random Data Pointt

In [26]:
def get_data(subject, training, PATH):
    '''
    Loads the dataset 2a of the BCI Competition IV
    available on http://bnci-horizon-2020.eu/database/data-sets

    Keyword arguments:
    subject -- number of subject in [1, .. ,9]
    training -- if True, load training data
                if False, load testing data

    Return: sorted_data_return 	numpy matrix 	size = NO_valid_trial x 22 x 1750
    sorted_class_return 	numpy matrix 	size = NO_valid_trial
    '''
    NO_channels = 22
    NO_tests = 6 * 48
    Window_Length = 7 * 250

    class_return = np.zeros(NO_tests)
    data_return = np.zeros((NO_tests, NO_channels, Window_Length))

    NO_valid_trial = 0
    print("PATH:", PATH, "subject:", subject)
    if training:
        mat_file = PATH + 'A0' + str(subject) + 'T.mat'
    else:
        mat_file = PATH + 'A0' + str(subject) + 'E.mat'

    a = sio.loadmat(mat_file)
    a_data = a['data'][0]  # Accessing the nested array

    for a_data1 in a_data:
        a_X = a_data1['X'][0, 0]
        a_trial = a_data1['trial'][0, 0]
        a_y = a_data1['y'][0, 0]
        a_artifacts = a_data1['artifacts'][0, 0]

        for trial in range(a_trial.size):
            if a_artifacts[trial] == 0:
                data_return[NO_valid_trial, :, :] = np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial]) + Window_Length), :22])
                class_return[NO_valid_trial] = int(a_y[trial])
                NO_valid_trial += 1

    # Sort the data based on class labels
    sorted_indices = np.argsort(class_return[:NO_valid_trial])
    sorted_data_return = data_return[sorted_indices]
    sorted_class_return = class_return[sorted_indices]

    # Update the original data with the sorted data
    for i, a_data1 in enumerate(a_data):
        a_data1['X'][0, 0] = np.transpose(sorted_data_return[i], (1, 0))
        a_data1['y'][0, 0] = sorted_class_return[i]

    # Save the updated data to the original .mat file
    sio.savemat('A03T_sorted.mat', {'data': a_data})

    return sorted_data_return, sorted_class_return

In [27]:
[sorted_data,sorted_class] = get_data(3,True,'C:/Users/jonadum/Desktop/Multiple Classes FBCSP/datasets/')

PATH: C:/Users/jonadum/Desktop/Multiple Classes FBCSP/datasets/ subject: 3


In [28]:
sorted_data

array([[[ 7.27539062e+00, -2.92968750e-01, -6.78710938e+00, ...,
          4.98046875e+00,  6.39648438e+00,  5.56640625e+00],
        [ 4.73632812e+00, -7.42187500e+00, -9.47265625e+00, ...,
          3.95507812e+00,  5.07812500e+00, -5.85937500e-01],
        [ 2.44140625e-01, -7.51953125e+00, -1.39160156e+01, ...,
          5.37109375e+00,  7.71484375e+00,  5.46875000e+00],
        ...,
        [ 1.01074219e+01, -1.46484375e-01, -2.00195312e+00, ...,
         -2.68554688e+00,  7.81250000e-01, -1.95312500e-01],
        [ 1.08886719e+01, -2.29492188e+00, -3.27148438e+00, ...,
         -2.19726562e+00,  1.85546875e+00,  1.80664062e+00],
        [ 9.22851562e+00,  2.44140625e-01,  1.12304688e+00, ...,
         -2.83203125e+00,  2.44140625e-01, -2.49023438e+00]],

       [[-1.25000000e+01,  4.88281250e-02, -1.84570312e+01, ...,
          7.76367188e+00,  1.36718750e+00,  1.25976562e+01],
        [-6.98242188e+00,  5.56640625e+00, -1.25000000e+01, ...,
          1.29882812e+01, -5.22460938e

In [29]:
sorted_class

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4.

In [30]:
def get_data(subject, training, PATH):
    '''
    Loads the dataset 2a of the BCI Competition IV
    available on http://bnci-horizon-2020.eu/database/data-sets

    Keyword arguments:
    subject -- number of subject in [1, .. ,9]
    training -- if True, load training data
                if False, load testing data

    Return: sorted_data_return 	numpy matrix 	size = NO_valid_trial x 22 x 1750
    sorted_class_return 	numpy matrix 	size = NO_valid_trial
    '''
    NO_channels = 22
    Window_Length = 7 * 250

    if training:
        mat_file = PATH + 'A0' + str(subject) + 'T.mat'
    else:
        mat_file = PATH + 'A0' + str(subject) + 'E.mat'

    a = sio.loadmat(mat_file)
    a_data = a['data'][0]  # Accessing the nested array

    data_return = []
    class_return = []

    for a_data1 in a_data:
        a_X = a_data1['X'][0, 0]
        a_trial = a_data1['trial'][0, 0]
        a_y = a_data1['y'][0, 0]
        a_artifacts = a_data1['artifacts'][0, 0]

        for trial in range(a_trial.size):
            if a_artifacts[trial] == 0:
                data_return.append(np.transpose(a_X[int(a_trial[trial]):(int(a_trial[trial]) + Window_Length), :22]))
                class_return.append(int(a_y[trial]))

    # Sort the data based on class labels
    sorted_indices = np.argsort(class_return)
    sorted_data_return = np.array(data_return)[sorted_indices]
    sorted_class_return = np.array(class_return)[sorted_indices]

    # Update the original data with the sorted data
    for i, a_data1 in enumerate(a_data):
        a_data1['X'][0, 0] = np.transpose(sorted_data_return[i], (1, 0))
        a_data1['y'][0, 0] = sorted_class_return[i]

    # Save the updated data to the original .mat file
    sio.savemat('test.mat', {'data': a_data})

    return sorted_data_return, sorted_class_return

In [31]:
[sorted_data,sorted_class] = get_data(3,True,'C:/Users/jonadum/Desktop/Multiple Classes FBCSP/datasets/')

In [32]:
sorted_data

array([[[ 7.27539062e+00, -2.92968750e-01, -6.78710938e+00, ...,
          4.98046875e+00,  6.39648438e+00,  5.56640625e+00],
        [ 4.73632812e+00, -7.42187500e+00, -9.47265625e+00, ...,
          3.95507812e+00,  5.07812500e+00, -5.85937500e-01],
        [ 2.44140625e-01, -7.51953125e+00, -1.39160156e+01, ...,
          5.37109375e+00,  7.71484375e+00,  5.46875000e+00],
        ...,
        [ 1.01074219e+01, -1.46484375e-01, -2.00195312e+00, ...,
         -2.68554688e+00,  7.81250000e-01, -1.95312500e-01],
        [ 1.08886719e+01, -2.29492188e+00, -3.27148438e+00, ...,
         -2.19726562e+00,  1.85546875e+00,  1.80664062e+00],
        [ 9.22851562e+00,  2.44140625e-01,  1.12304688e+00, ...,
         -2.83203125e+00,  2.44140625e-01, -2.49023438e+00]],

       [[-1.25000000e+01,  4.88281250e-02, -1.84570312e+01, ...,
          7.76367188e+00,  1.36718750e+00,  1.25976562e+01],
        [-6.98242188e+00,  5.56640625e+00, -1.25000000e+01, ...,
          1.29882812e+01, -5.22460938e

In [33]:
sorted_class

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4])